In [19]:
import boto3
import pandas as pd
from io import StringIO
import os, glob
from datetime import datetime, timedelta

In [20]:
arg_date = '2021-03-30'

In [21]:
arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)

In [22]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('xetra-1234')



In [23]:
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d').date() >= arg_date_dt]

KeyboardInterrupt: 

In [ ]:
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR10.csv'),
 s3.Object

In [ ]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')

In [ ]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [ ]:
df_all = pd.DataFrame(columns=df_init.columns)
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_all = df_all.append(df, ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
df

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,CA0679011084,ABR,BARRICK GOLD CORP.,Common stock,EUR,2504196,2021-03-15,15:00,17.060,17.060,17.060,17.060,2000,1
1,DE000A0BVU28,OSP2,USU SOFTWARE AG,Common stock,EUR,2504280,2021-03-15,15:00,26.600,26.600,26.600,26.600,70,1
2,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2021-03-15,15:00,22.120,22.120,22.120,22.120,119,1
3,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2021-03-15,15:00,204.300,204.400,204.300,204.400,98,4
4,DE000A0F5UJ7,EXV1,IS.S.EU.600 BANK.U.ETF A.,ETF,EUR,2504306,2021-03-15,15:00,12.656,12.662,12.656,12.662,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13032,IE00BMC38736,VVSM,VANE.SEMICONDUCTOR UC.ETF,ETF,EUR,5879481,2021-03-15,15:59,18.500,18.500,18.500,18.500,45,1
13033,DE000A3H2200,NA9,NAGARRO SE NA O.N.,Common stock,EUR,5906903,2021-03-15,15:59,87.200,87.200,87.200,87.200,5,1
13034,US36467W1099,GS2C,GAMESTOP CORP. A,Common stock,EUR,5990151,2021-03-15,15:59,210.500,210.650,210.500,210.650,339,3
13035,DE000A2LQ884,AG1,AUTO1 GROUP SE INH O.N.,Common stock,EUR,6080964,2021-03-15,15:59,46.790,46.790,46.790,46.790,196,1


In [ ]:
df_all.head(10)

In [ ]:
columns = ["ISIN", "Mnemonic", "Date", "Time", "StartPrice", "EndPrice", "MinPrice", "MaxPrice", "TradedVolume"]
df_all = df_all.loc[:, columns]

In [ ]:
df_all

In [ ]:
df_all.dropna(inplace=True)

In [ ]:
df_all.shape

## Get opening price per ISIN and day

In [ ]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

In [ ]:
df_all[df_all['ISIN']=='DE0007568578']

## Get closing price per ISIN and day


In [ ]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

In [ ]:
df_all[df_all['ISIN']=='DE0007568578']

## Aggregations

In [ ]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

In [ ]:
df_all

# percent change prev closing

In [ ]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [ ]:
df_all

In [ ]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [ ]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [ ]:
df_all = df_all.round(decimals=2)

In [ ]:
df_all.reset_index(inplace=True)

In [ ]:
df_all = df_all[df_all.Date >= arg_date]

In [ ]:
df_all